In [136]:
import numpy as np
import pandas as pd
import json

In [ ]:
df_train = pd.read_json("../data/raw/train.json")
print(df_train.head(3))

In [185]:
#grouping by authors
a = df_train.groupby(['authorName']).size().reset_index().sort_values(0,ascending=False)
print(a)

            authorName   0
835          Chuhan Wu  13
3926    Ryan Cotterell  13
1889        Ivan Vulic  13
3789     Ramit Sawhney  12
3991        S. Malmasi  12
...                ...  ..
1535    Gonzalo Méndez   1
3742    R. Rajalakshmi   1
1534  Gonzalo Iglesias   1
3744       R. SarathP.   1
2755         M. Becker   1

[5511 rows x 2 columns]


Chuhan wu is most common, make a test model for her

In [139]:
df_train_chuhan = df_train[df_train['authorName']=='Chuhan Wu'] 

In [ ]:
df_train_chuhan

In [ ]:
df_train_chuhan_NLP = df_train_chuhan[['title', 'abstract']]
df_train_chuhan_NLP

In [190]:
df_train_chuhan_place = df_train_chuhan[['year', 'venue']]
df_train_chuhan_place

,year,venue
655,2021,FINDINGS
1934,2019,NAACL
2815,2018,Fig-Lang@NAACL-HLT
5175,2018,*SEMEVAL
5311,2021,FINDINGS
5576,2018,*SEMEVAL
6271,2019,EMNLP
8461,2022,ACL
8721,2019,EMNLP
9844,2018,EMNLP 2018


## 1 - We use bag-of-words technique on our first model 

In [ ]:
# selected the first abstract
df_train_chuhan_NLP_abst1 = [df_train_chuhan_NLP.iloc[0,1]]
df_train_chuhan_NLP_abst1

In [144]:
from sklearn.feature_extraction.text import CountVectorizer

In [145]:
vectorizer = CountVectorizer(input='content')

# our sample set is df_train_chuhan_NLP_abst1
# fit the bag-of-words model

bag = vectorizer.fit_transform(df_train_chuhan_NLP_abst1)

#### Get unique words / tokens found in all the documents. The unique words / tokens represents the features

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
# Associate the indices with each unique word

print(vectorizer.vocabulary_)

In [148]:
# Print the numerical feature vector
print(bag.toarray())

[[7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 4 1 1 1 1 1 1 1 6 2 1 1 1 1
  1 1 1 6 1 1 1 1 1 1 1 1 3 1 2 1 1 7 1 3 1 1 1 2 1 2 1 1 1 1 6 6 3 2 1 1
  1 1 1 1 2 1 6 1 1 2 4 1 1 1 8 2 3 1 1]]


Now test with the second abstract

In [ ]:
df_train_chuhan_NLP_abst2 = [df_train_chuhan_NLP.iloc[1,1]]
df_train_chuhan_NLP_abst2

In [151]:
testbag = vectorizer.transform(df_train_chuhan_NLP_abst2)
print(testbag.toarray())
#print(testbag)

[[10  0  1  2  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  2  1  0  0  1
   0  3  1  1  1  0  4  6  0  0  0  0  0  1  0  3  0  0  1  0  0  0  0  0
   0  3  0  0  0  0  0  2  2  0  0  3  1  0  0  0  1  0  0  0  1  1  0  1
   0  0  0  0  1  0  7  0  0  1  7  0  0  0  8  0  3  0  2]]


In [152]:

print(testbag.count_nonzero())

32


In [131]:
print(bag.count_nonzero())

91


In [133]:
score = testbag.count_nonzero()/ bag.count_nonzero()
print(score)

0.3516483516483517


### 2 - Now we use all available texts to build the model

In [ ]:
# Make a list with all abstracts and titles
df_train_chuhan_NLP_abstracts = df_train_chuhan_NLP.iloc[:,1].tolist()
df_train_chuhan_NLP_titles = df_train_chuhan_NLP.iloc[:,0].tolist()
df_train_chuhan_NLP_all = df_train_chuhan_NLP_titles + df_train_chuhan_NLP_abstracts

print(df_train_chuhan_NLP_all)

In [172]:
vectorizer_all = CountVectorizer(input='content')

# our sample set is df_train_chuhan_NLP_abst1
# fit the bag-of-words model

bag = vectorizer_all.fit_transform(df_train_chuhan_NLP_all)

In [ ]:
print(vectorizer_all.get_feature_names())

In [ ]:
test_Same_author = df_train_chuhan_NLP.iloc[5,:].tolist()
test_Same_author

In [182]:

bag_test_Same_author = vectorizer_all.transform(test_Same_author)

In [184]:
score = bag_test_Same_author.count_nonzero()/ bag.count_nonzero()
print(score)

0.08921644685802949


#### testing with another author Ryan Cotterell

In [ ]:
df_train_ryan = df_train[df_train['authorName']=='Ryan Cotterell']
df_train_ryan_NLP = df_train_ryan[['title', 'abstract']]
ryan_sent1 = df_train_ryan_NLP.iloc[6,:].tolist()
print(ryan_sent1)

In [189]:
bag_ryan = vectorizer_all.transform(ryan_sent1)
score2 = bag_ryan.count_nonzero()/ bag.count_nonzero()
print(score2)

0.03413498836307215


In [191]:
bag_ryan.count_nonzero()


44

In [ ]:
bag_test_Same_author2 = vectorizer_all.transform(test_Same_author2)
score2 = bag_test_Same_author2.count_nonzero()/ bag.count_nonzero()
print(score2)